In [64]:
#Carga de Librerias
#pandas Visualizacion de datos
#numpy Analisis matematico y cientifico de datos {arrays}
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # Suprimir Warnings
##Librerias para el algoritmo
from kmodes.kmodes import KModes
##Libreria para la grafica
import matplotlib.pyplot as plt
%matplotlib inline
##Para el manejo de las fechas
from datetime import datetime
##Para el procesamiento de los datos
from sklearn import preprocessing

In [65]:
df=pd.read_csv('C:/Users/PC/Desktop/Tesis/Practicum4_SistemasInteligentes_UTPL/DataSet/Dataset.csv',encoding='ISO-8859-1')

In [66]:
df.head()

,id,case_in_country,reporting date,location,country,gender,age,symptom_onset,hosp_visit_date,exposure_start,exposure_end,visiting Wuhan,from Wuhan,death,recovered,symptom
0,1,0,1/20/2020,Shenzhen Guangdong,China,male,66.0,1/3/20,1/11/20,12/29/2019,1/4/20,1,0,0,0,NaN
1,2,0,1/20/2020,Shanghai,China,female,56.0,1/15/2020,1/15/2020,NaN,1/12/20,0,1,0,0,NaN
2,3,0,1/21/2020,Zhejiang,China,male,46.0,1/4/20,1/17/2020,NaN,1/3/20,0,1,0,0,NaN
3,4,0,1/21/2020,Tianjin,China,female,60.0,NaN,1/19/2020,NaN,NaN,1,0,0,0,NaN
4,5,0,1/21/2020,Tianjin,China,male,58.0,NaN,1/14/2020,NaN,NaN,0,0,0,0,NaN


In [67]:
#Renombrar Columnas para mas facil tratamiento del dataset
df.rename(columns={'case_in_country':'caseInCountry','reporting date': 'reportingDate','symptom_onset':'symptomOnset', 'hosp_visit_date':'hospVisitDate','exposure_start':'exposureStart','exposure_end':'exposureEnd','visiting Wuhan': 'visitingWuhan','from Wuhan': 'fromWuhan'}, inplace=True)

In [68]:
#Eliminar los datos vacios de las columnas genero y edad.
df=df[(df["gender"].notnull())&(df["age"].notnull())]

In [69]:
#Hacer una copia del DateFrame
dfCopy = df.copy()

In [70]:
#Eliminar Columnas Pruebas
#dfCopy.head()
#dfCopy.id.count()
#df.id.count()

In [71]:
#Datos Categoricos a la columna genero [gender]
dfCopy['gender'].replace(['female','male'],[2,1],inplace=True)

In [72]:
# Agrupar la columna edad
dfCopy['ageClu'] = pd.cut(dfCopy['age'], [0, 20, 30, 40, 50, 60, 70, 80, 90, 100], 
labels=['0-20', '20-30', '30-40', '40-50','50-60','60-70','70-80', '80-90','90-100'])

In [73]:
#Eliminar Columna age
dfCopy  = dfCopy.drop('age',axis = 1)

In [74]:
dfCopy.head()

,id,caseInCountry,reportingDate,location,country,gender,symptomOnset,hospVisitDate,exposureStart,exposureEnd,visitingWuhan,fromWuhan,death,recovered,symptom,ageClu
0,1,0,1/20/2020,Shenzhen Guangdong,China,1,1/3/20,1/11/20,12/29/2019,1/4/20,1,0,0,0,NaN,60-70
1,2,0,1/20/2020,Shanghai,China,2,1/15/2020,1/15/2020,NaN,1/12/20,0,1,0,0,NaN,50-60
2,3,0,1/21/2020,Zhejiang,China,1,1/4/20,1/17/2020,NaN,1/3/20,0,1,0,0,NaN,40-50
3,4,0,1/21/2020,Tianjin,China,2,NaN,1/19/2020,NaN,NaN,1,0,0,0,NaN,50-60
4,5,0,1/21/2020,Tianjin,China,1,NaN,1/14/2020,NaN,NaN,0,0,0,0,NaN,50-60


In [75]:
#Desglosar Symptom
#Columna 1
dfCopy["fever"]=((dfCopy["symptom"].str.contains('fever',na=False,regex=False))*1)

In [76]:
#df_New[300:310]

In [77]:
#Agregar columna 2
dfCopy["cough"]=((dfCopy["symptom"].str.contains('cough',na=False,regex=False))*1)

In [78]:
#Agregar columna 3
dfCopy["vomiting_nausea_fatigue"]=((dfCopy["symptom"].str.contains('vomiting|nausea|fatigue',na=False,regex=True))*1)

In [79]:
##Agregar columna 4
dfCopy["difficultBreathing_respiratoryDistress"]=(dfCopy["symptom"].str.contains('breath|distress',na=False,regex=True))*1

In [80]:
##Agregar columna 5
dfCopy["headache_heavyHead"]=((dfCopy["symptom"].str.contains('head',na=False,regex=False))*1)

In [81]:
##Agregar columna 6
dfCopy["throat"]=((dfCopy["symptom"].str.contains('throat',na=False,regex=False))*1)

In [82]:
##Agregar columna 7
dfCopy["chill_cold"]=((dfCopy["symptom"].str.contains('chill|cold',na=False, regex=True))*1)

In [83]:
##Agregar columna 8
dfCopy["otherSymptom"]=((dfCopy["symptom"].str.contains('joint|runny|diarrhea|abdominal|pneumonia|loss|malaise|sputum|muscle|breathlessness|dyspnea|loss|nasal|mild|sneeze|chest|thirst|flu|cramps|aching|body|aches|reflux|physical|tired|myalgias',na=False,regex=True))*1)

In [84]:
##Asintomatico
dfCopy["asymptomatic"]=((dfCopy["symptom"].str.contains('SinSintomas',na=True,regex=True))*1)


In [85]:
#Eliminacion columna desglosada
dfCopy.drop(['symptom'], axis=1, inplace=True)

In [86]:
dfCopy.head()

,id,caseInCountry,reportingDate,location,country,gender,symptomOnset,hospVisitDate,exposureStart,exposureEnd,...,ageClu,fever,cough,vomiting_nausea_fatigue,difficultBreathing_respiratoryDistress,headache_heavyHead,throat,chill_cold,otherSymptom,asymptomatic
0,1,0,1/20/2020,Shenzhen Guangdong,China,1,1/3/20,1/11/20,12/29/2019,1/4/20,...,60-70,0,0,0,0,0,0,0,0,1
1,2,0,1/20/2020,Shanghai,China,2,1/15/2020,1/15/2020,NaN,1/12/20,...,50-60,0,0,0,0,0,0,0,0,1
2,3,0,1/21/2020,Zhejiang,China,1,1/4/20,1/17/2020,NaN,1/3/20,...,40-50,0,0,0,0,0,0,0,0,1
3,4,0,1/21/2020,Tianjin,China,2,NaN,1/19/2020,NaN,NaN,...,50-60,0,0,0,0,0,0,0,0,1
4,5,0,1/21/2020,Tianjin,China,1,NaN,1/14/2020,NaN,NaN,...,50-60,0,0,0,0,0,0,0,0,1


In [87]:
dfCopy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 825 entries, 0 to 1084
Data columns (total 24 columns):
 #   Column                                  Non-Null Count  Dtype   
---  ------                                  --------------  -----   
 0   id                                      825 non-null    int64   
 1   caseInCountry                           825 non-null    int64   
 2   reportingDate                           825 non-null    object  
 3   location                                825 non-null    object  
 4   country                                 825 non-null    object  
 5   gender                                  825 non-null    int64   
 6   symptomOnset                            546 non-null    object  
 7   hospVisitDate                           493 non-null    object  
 8   exposureStart                           104 non-null    object  
 9   exposureEnd                             298 non-null    object  
 10  visitingWuhan                           825 non-n

In [88]:
#Seleccionar columnas a hacerla categoricas en un nuevo dataframe
dfCategorical = dfCopy[['country','ageClu']]

In [92]:
# Eliminar Columnas Copiadas
dfCopy.drop(['country','ageClu'], axis=1, inplace=True)

,id,caseInCountry,reportingDate,location,gender,symptomOnset,hospVisitDate,exposureStart,exposureEnd,visitingWuhan,...,recovered,fever,cough,vomiting_nausea_fatigue,difficultBreathing_respiratoryDistress,headache_heavyHead,throat,chill_cold,otherSymptom,asymptomatic
0,1,0,1/20/2020,Shenzhen Guangdong,1,1/3/20,1/11/20,12/29/2019,1/4/20,1,...,0,0,0,0,0,0,0,0,0,1
1,2,0,1/20/2020,Shanghai,2,1/15/2020,1/15/2020,NaN,1/12/20,0,...,0,0,0,0,0,0,0,0,0,1
2,3,0,1/21/2020,Zhejiang,1,1/4/20,1/17/2020,NaN,1/3/20,0,...,0,0,0,0,0,0,0,0,0,1
3,4,0,1/21/2020,Tianjin,2,NaN,1/19/2020,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,1
4,5,0,1/21/2020,Tianjin,1,NaN,1/14/2020,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,1


In [93]:
#Hacer datos Categoricos las columnas 'country' y 'ageClu
le = preprocessing.LabelEncoder()
dfCategorical = dfCategorical.apply(le.fit_transform)
dfCategorical.head()

,country,ageClu
0,3,5
1,3,4
2,3,3
3,3,4
4,3,4


In [94]:
##Unir el dataframe copia con el dataframe categorico
dfNew = pd.concat([dfCopy, dfCategorical], axis=1)
dfNew.head()

,id,caseInCountry,reportingDate,location,gender,symptomOnset,hospVisitDate,exposureStart,exposureEnd,visitingWuhan,...,cough,vomiting_nausea_fatigue,difficultBreathing_respiratoryDistress,headache_heavyHead,throat,chill_cold,otherSymptom,asymptomatic,country,ageClu
0,1,0,1/20/2020,Shenzhen Guangdong,1,1/3/20,1/11/20,12/29/2019,1/4/20,1,...,0,0,0,0,0,0,0,1,3,5
1,2,0,1/20/2020,Shanghai,2,1/15/2020,1/15/2020,NaN,1/12/20,0,...,0,0,0,0,0,0,0,1,3,4
2,3,0,1/21/2020,Zhejiang,1,1/4/20,1/17/2020,NaN,1/3/20,0,...,0,0,0,0,0,0,0,1,3,3
3,4,0,1/21/2020,Tianjin,2,NaN,1/19/2020,NaN,NaN,1,...,0,0,0,0,0,0,0,1,3,4
4,5,0,1/21/2020,Tianjin,1,NaN,1/14/2020,NaN,NaN,0,...,0,0,0,0,0,0,0,1,3,4


In [95]:
dfNew.id.count()

825

In [98]:
# Bajarno el dataFrame a .CSV
dfNew.to_csv('C:/Users/PC/Desktop/Prueba/PruebasSabado.csv')

In [100]:
dfNew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 825 entries, 0 to 1084
Data columns (total 24 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   id                                      825 non-null    int64 
 1   caseInCountry                           825 non-null    int64 
 2   reportingDate                           825 non-null    object
 3   location                                825 non-null    object
 4   gender                                  825 non-null    int64 
 5   symptomOnset                            546 non-null    object
 6   hospVisitDate                           493 non-null    object
 7   exposureStart                           104 non-null    object
 8   exposureEnd                             298 non-null    object
 9   visitingWuhan                           825 non-null    int64 
 10  fromWuhan                               825 non-null    int64 
 11  death

In [101]:
##Crear un nuevo dateframe para aplicar el algoritmo Kmeans
dfClustering = dfNew[['gender','visitingWuhan','fromWuhan','death','recovered','fever','cough','vomiting_nausea_fatigue','difficultBreathing_respiratoryDistress','headache_heavyHead','throat','chill_cold','otherSymptom','asymptomatic','country','ageClu']]

In [102]:
dfClustering.head()

,gender,visitingWuhan,fromWuhan,death,recovered,fever,cough,vomiting_nausea_fatigue,difficultBreathing_respiratoryDistress,headache_heavyHead,throat,chill_cold,otherSymptom,asymptomatic,country,ageClu
0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,3,5
1,2,0,1,0,0,0,0,0,0,0,0,0,0,1,3,4
2,1,0,1,0,0,0,0,0,0,0,0,0,0,1,3,3
3,2,1,0,0,0,0,0,0,0,0,0,0,0,1,3,4
4,1,0,0,0,0,0,0,0,0,0,0,0,0,1,3,4


In [ ]:
#Using K-Mode with "Huang" initialization
km_huang = KModes(n_clusters=2, init = "Huang", n_init = 1, verbose=1)
fitClusters_huang = km_huang.fit_predict(df_Clustering)

In [ ]:
# Predicted clusters
fitClusters_huang


In [ ]:
cost = []
for num_clusters in list(range(1,5)):
    kmode = KModes(n_clusters=num_clusters, init = "Cao", n_init = 1, verbose=1)
    kmode.fit_predict(df_Clustering)
    cost.append(kmode.cost_)

In [ ]:
y = np.array([i for i in range(1,5,1)])
plt.plot(y,cost)

In [ ]:
## Choosing K=2
km_cao = KModes(n_clusters=2, init = "Cao", n_init = 1, verbose=1)
fitClusters_cao = km_cao.fit_predict(df_Clustering)

In [ ]:
fitClusters_cao

In [ ]:
#Combining the predicted clusters with the original DF
df_Clustering = df_New_copy.reset_index()

In [ ]:
clustersDf = pd.DataFrame(fitClusters_cao)
clustersDf.columns = ['cluster_predicted']
combinedDf = pd.concat([df_Clustering, clustersDf], axis = 1).reset_index()
combinedDf = combinedDf.drop(['index', 'level_0'], axis = 1)

In [ ]:
combinedDf.head()

In [ ]:
#Cluster Identification
cluster_0 = combinedDf[combinedDf['cluster_predicted'] == 0]
cluster_1 = combinedDf[combinedDf['cluster_predicted'] == 1]

In [ ]:
# Job
plt.subplots(figsize = (15,5))
sns.countplot(x=combinedDf['fever'],order=combinedDf['fever'].value_counts().index,hue=combinedDf['cluster_predicted'])
plt.show()